In [ ]:
import requests as r
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date, timezone
import plotly.express as px
import plotly.graph_objects as go

endpoint_eth = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
endpoint_op = 'https://api.thegraph.com/subgraphs/name/ianlapham/optimism-post-regenesis'
endpoint_arbi = 'https://api.thegraph.com/subgraphs/name/ianlapham/arbitrum-dev'
endpoint_poly = 'https://api.thegraph.com/subgraphs/name/ianlapham/uniswap-v3-polygon'
#headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:32.0) Gecko/20100101 Firefox/32.0',}

trailing_days = 365
chain = 'optimism'

In [ ]:


if chain =='ethereum':
        endpoint = endpoint_eth
        strf = 'eth'
elif chain =='optimism':
        endpoint = endpoint_op
        strf = 'op'
elif chain =='arbitrum':
        endpoint = endpoint_arbi
        strf = 'arbi'

start_date = date.today()-timedelta(days=trailing_days +1)
end_date = date.today()

# start_dt = start_date.timestamp()
start_dt = int( ( start_date - date(1970,1,1) ) / timedelta(seconds=1) )
end_dt = int( ( end_date - date(1970,1,1) ) / timedelta(seconds=1) )

print(start_date)
print(start_dt)
print(end_dt)

In [ ]:
# skip = 0
# first = 1000
last_id = ''
dflist = []
is_response = 1

while is_response == 1:
  qlq = """
  {
    pools(first:1000@id_gt@) {
      id
      token0 {
        symbol
      }
      token1 {
        symbol
      }
      feeTier
      txCount
      feesUSD
      totalValueLockedUSD
      poolDayData(where:{date_gt:@start_dt@, date_lt:@end_dt@}) {
        id
        date
        volumeUSD
        feesUSD
      }
    }

  }
  """
  #feesUSD_gt:0, orderBy: totalValueLockedUSD, orderDirection: desc
  #, orderBy: id, orderDirection: asc
  # qlq = qlq.replace('@first@',str(first))
  # qlq = qlq.replace('@skip@',str(skip))
  if last_id =='':
    qlq = qlq.replace('@id_gt@' , '')
  else:
    qlq = qlq.replace('@id_gt@' , ', where: {id_gt:\"@last_id@\"}')
    qlq = qlq.replace('@last_id@' , last_id)

  qlq = qlq.replace('@start_dt@',str(start_dt))
  qlq = qlq.replace('@end_dt@',str(end_dt))

  response = r.post(endpoint, json={'query': qlq})
  # print(response)
    
  if response.status_code != 200:
    is_response = 0
  else:
    try:
      rjs = pd.DataFrame(response.json()['data']['pools'])
      dflist.append(rjs)
      last_id = rjs.iloc[-1]['id']
      print(last_id)
      # print(skip)
      # skip = skip + first
    except:
      break

In [ ]:
print(response)
# print(qlq)
# print(dflist)

In [ ]:
# data = {"query" : qlq}
# response = r.post(url=endpoint, headers=headers, data=data)


# print(rjs)
df = pd.concat(dflist)
print(len(df))
df.reset_index(inplace=True,drop=True)

df['totalValueLockedUSD'] = df['totalValueLockedUSD'].astype(np.float64)
df['feesUSD'] = df['feesUSD'].astype(np.float64)
df['feeTier'] = df['feeTier'].astype(np.float64)




In [ ]:
display(df['poolDayData'])

In [ ]:
# df = pd.DataFrame(df).head(3)

for index, row in df.iterrows():
    tdf = pd.DataFrame(row['poolDayData'])
    try:
        # print(tdf)
        tdf['feesUSD'] = tdf['feesUSD'].astype(np.float64)
        tdf['volumeUSD'] = tdf['volumeUSD'].astype(np.float64)
        # print('weee')
        fees_90 = tdf['feesUSD'].sum()
        volume_90  = tdf['volumeUSD'].sum()
        # display(tdf)
        # print(fees_90)
        df.loc[index,'fees_usd_t90d'] = fees_90
        df.loc[index,'volume_usd_t90d'] = volume_90
    except:
        # print(tdf)
        continue

# df3 = pd.json_normalize(df3['poolDayData'])
print("total volume " + str(df['volume_usd_t90d'].sum()))
# df['totalValueLockedUSD'] = df['totalValueLockedUSD'].astype(np.float64)
# df['feesUSD'] = df['feesUSD'].astype(np.float64)
# df['feeTier'] = df['feeTier'].astype(np.float64)
# pd.json_normalize(df['poolDayData'])
df = df[df['totalValueLockedUSD'] > 0]
display(df.sort_values('totalValueLockedUSD', ascending = False))

In [ ]:
#exclude known spam pair
# print(len(df))
# print(df['totalValueLockedUSD'].sum())
df = df[df['id'] != '0xa850478adaace4c08fc61de44d8cf3b64f359bec'] #UMIIE/UMIIE2
# print(df['totalValueLockedUSD'].sum())

df = df[df['totalValueLockedUSD'] > 0]
# display(df)
# print(len(df))

total_tvl = df['totalValueLockedUSD'].sum()
#filter df to only fee-generating pools
df = df[df['feesUSD']>0]
# print(len(df))
fee_generating_tvl = df['totalValueLockedUSD'].sum()
print(total_tvl)
print(fee_generating_tvl)
df.drop(columns=['poolDayData','txCount','feesUSD'],inplace=True)

In [ ]:
df.reset_index(inplace=True,drop=True)
# df['token0'] = df['token0'].apply(lambda x: pd.json_normalize(x))
df['token0'] = pd.json_normalize(df['token0'])
df['token1'] = pd.json_normalize(df['token1'])
df['feeTier'] = df['feeTier'] / 1e6

# df = df.json_normalize()

# total_tvl = 12_660_000_000 #12.6 bil - eventually find how to actually pull this
# total_tvl = 5_520_000_000 #12.6 bil - eventually find how to actually pull this
df['tvl_share'] = df['totalValueLockedUSD'] / total_tvl
df['pool'] = df['token0'] + '/' + df['token1'] + ' - ' + (df['feeTier'].astype(np.float64)*100).astype(str) + '%'
display(df.sort_values('totalValueLockedUSD', ascending = False))


In [ ]:
df['fee_apr_plus10m'] = ( df['fees_usd_t90d']*(365/90) ) / (df['totalValueLockedUSD'] + 10_000_000)
df.fillna(0,inplace=True)
display(df.sort_values('totalValueLockedUSD', ascending = False))
df = df.sort_values('totalValueLockedUSD', ascending = False)
df.reset_index(inplace=True,drop=True)
df.to_csv('uni_tvl_share' + strf + '.csv')

In [ ]:
fig_app = px.treemap(df[df['fees_usd_t90d']>100], \
                        path=[px.Constant("all"), 'pool'], \
                 values='totalValueLockedUSD', color='fee_apr_plus10m'#color='fees_usd_t90d'
#                 ,color_discrete_map={'-1':'red', '1':'green'})
                ,color_continuous_scale='BuGn'
                )
# fig.data[0].textinfo = 'label+text+value'
fig_app.update_traces(root_color="lightgrey")
fig_app.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig_app.show()